In [1]:
import numpy as np
import torch
import sounddevice as sd
import os
from pygame import mixer
import torchaudio
from scipy.interpolate import interp1d
from sklearn.metrics import roc_curve
from torch.nn.utils import clip_grad_norm_
from scipy.optimize import brentq
from torch import nn
import numpy as np
import torch
from matplotlib import cm
from pathlib import Path
import numpy as np
from scipy.ndimage.morphology import binary_dilation
from pathlib import Path
from typing import Optional, Union
from warnings import warn
import numpy as np
import librosa
import struct
import librosa
import librosa.filters
import numpy as np
from scipy import signal
from scipy.io import wavfile
import soundfile as sf
from pathlib import Path
from typing import Union, List
import numpy as np
import librosa
import ast
import pprint
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import time
import numpy as np
import sys
from collections import namedtuple
import time
from scipy.signal import lfilter
import webrtcvad

C:\Users\gabri\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\gabri\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\gabri\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


pygame 2.1.2 (SDL 2.0.18, Python 3.9.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\gabri\AppData\Local\Temp\ipykernel_1984\2159315701.py:17: DeprecationWarning: Please use `binary_dilation` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import binary_dilation


In [8]:
encoder_path = 'Audio_Generation/Generation_Scripts/saved_models/default/encoder.pt'
synth_path = 'Audio_Generation/Generation_Scripts/saved_models/default/synthesizer.pt'
vocode_path = 'Audio_Generation/Generation_Scripts/saved_models/default/vocoder.pt'

In [9]:
# Create a new object
obj = Audio_Obj(encoder_path, synth_path, vocode_path)

In [17]:
# Load in files
p = "Audio_Generation/Generation_Scripts/data/albedo"
obj.load_from_browser("1.5.mp3", p)
# obj.load_from_browser("2.5.mp3", p)
# obj.load_from_browser("3.5.mp3", p)
# obj.load_from_browser("4.5.mp3", p)
# obj.load_from_browser("5.5.mp3", p)
# obj.load_from_browser("6.5.mp3", p)
# obj.load_from_browser("7.5.mp3", p)
# obj.load_from_browser("8.5.mp3", p)

C:\Users\gabri\AppData\Local\Temp\ipykernel_1984\4163032397.py:823: FutureWarning: Pass sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  wav = librosa.load(str(fpath), hparams.sample_rate)[0]


Loaded 1.5.mp3
Loading the encoder Audio_Generation/Generation_Scripts/saved_models/default/encoder.pt... 


NameError: name 'mel_n_channels' is not defined

In [46]:
text = "hello"

In [47]:
# Create the audio
obj.synthesize(text)
obj.vocode()

Generating the mel spectrogram...
Loading the synthesizer Audio_Generation/Generation_Scripts/saved_models/default/synthesizer.pt... 
Synthesizer using device: cuda
Done
Trainable Parameters: 30.870M
Loaded synthesizer "Audio_Generation/Generation_Scripts/saved_models/default/synthesizer.pt" trained to step 295000


NameError: name 'cleaners' is not defined

In [2]:




# My stuff
class str_enc():
    def __init__(self, string):
        self.name = string
    def __str__(self):
        return self.name




class Audio_Obj:
    def __init__(self, encoder_path, synth_path, vocode_path):
        # Important paths
        self.model_fpath = encoder_path
        self.synth_path = synth_path
        self.vocode_path = vocode_path

        # Synthesizer is initially NOne
        self.synthesizer = None

        # The embeddings are initially nothing
        self.embed = None

        # Utterances are the inputted audio clips
        self.utterances = set()






    # Load a sound file in
    # name - filename to load in
    # speaker_name - Path to file
    def load_from_browser(self, name, speaker_name):
        # Get the wav from the disk. We take the wav with the vocoder/synthesizer format for
        # playback, so as to have a fair comparison with the generated audio
        fpath = os.path.abspath(speaker_name+os.sep+name)
        wav = Synthesizer.load_preprocess_wav(fpath)
        print("Loaded %s" % name)

        self.add_real_utterance(wav, name, speaker_name)

    # Add an utterance
    def add_real_utterance(self, wav, name, speaker_name):
        # Compute the embedding
        if not encoder.is_loaded():
            self.init_encoder()
        encoder_wav = encoder.preprocess_wav(wav)
        embed, partial_embeds, _ = encoder.embed_utterance(encoder_wav, return_partials=True)

        # Save the current complete embedding
        self.embed = embed

        # Add the utterance
        spec = None # Spec not needed. Only needed for vis
        utterance = Utterance(name, speaker_name, wav, spec, embed, partial_embeds, False)
        self.utterances.add(utterance)

    # Initialize the encoder model to encode given audio clips
    def init_encoder(self):
        print("Loading the encoder %s... " % self.model_fpath)
        encoder.load_model(str_enc(self.model_fpath), device=torch.device("cpu"))
        print("Done")








    # This is where inferencing happens
    def synthesize(self, text, random_seed=None):
        # Make sure embeddings have been added
        assert self.embed is not None, "Make sure to add at least one embedding"

        print("Generating the mel spectrogram...")

        # Update the synthesizer random seed
        if random_seed != None:
            seed = int(random_seed)
        else:
            seed = None
        if seed is not None:
            torch.manual_seed(seed)

        # Synthesize the spectrogram
        if self.synthesizer is None or seed is not None:
            self.init_synthesizer()

        texts = text.split("\n")
        embeds = [self.embed] * len(texts)
        specs = self.synthesizer.synthesize_spectrograms(texts, embeds)
        breaks = [spec.shape[1] for spec in specs]
        spec = np.concatenate(specs, axis=1)

        # Save the currently generated utterance
        self.current_generated = ("", spec, breaks, None)


    # Initialize the synthesizer model to output audio
    # spectograms
    def init_synthesizer(self):
        model_fpath = str_enc(self.synth_path)

        print("Loading the synthesizer %s... " % model_fpath)
        self.synthesizer = Synthesizer(model_fpath)
        print("Done")






    # Used to do the second part of sysnthesis with the
    # synthesizer data
    # random_seed - Seed to initialize the model gneeration
    # traim_silences - True to trim excessive silences, False otherwise
    def vocode(self, random_seed=None, trim_silences=False):
        # Get the currently generated spectogram
        speaker_name, spec, breaks, _ = self.current_generated
        assert spec is not None

        # Initialize the vocoder model and make it determinstic, if user provides a seed
        if random_seed != None:
            seed = int(random_seed)
        else:
            seed = None
        if seed is not None:
            torch.manual_seed(seed)

        # Synthesize the waveform
        if not vocoder.is_loaded() or seed is not None:
            self.init_vocoder()

        # Generate a waveform from the spectogram
        def vocoder_progress(i, seq_len, b_size, gen_rate):
            real_time_factor = (gen_rate / Synthesizer.sample_rate) * 1000
            line = "Waveform generation: %d/%d (batch size: %d, rate: %.1fkHz - %.2fx real time)" \
                    % (i * b_size, seq_len * b_size, b_size, gen_rate, real_time_factor)
        if self.vocode_path is not None:
            wav = vocoder.infer_waveform(spec, progress_callback=vocoder_progress, fade=False)
        else:
            print("Waveform generation with Griffin-Lim... ")
            wav = Synthesizer.griffin_lim(spec)
        print(" Done!", "append")

        # Add breaks
        b_ends = np.cumsum(np.array(breaks) * Synthesizer.hparams.hop_size)
        b_starts = np.concatenate(([0], b_ends[:-1]))
        wavs = [wav[start:end] for start, end, in zip(b_starts, b_ends)]
        breaks = [np.zeros(int(0.15 * Synthesizer.sample_rate))] * len(breaks)
        wav = np.concatenate([i for w, b in zip(wavs, breaks) for i in (w, b)])

        # Trim excessive silences
        if trim_silences == True:
            wav = encoder.preprocess_wav(wav)

        # Play the audio
        wav = torch.tensor(wav/np.abs(wav).max() * 0.97, dtype=torch.float32).unsqueeze(0)
        torchaudio.save("tmp.mp3", wav, Synthesizer.sample_rate)
        mixer.init()
        mixer.music.load('tmp.mp3')
        mixer.music.play()

        # # Compute the embedding
        # # TODO: this is problematic with different sampling rates, gotta fix it
        # if not encoder.is_loaded():
        #     self.init_encoder()
        # encoder_wav = encoder.preprocess_wav(wav)
        # embed, partial_embeds, _ = encoder.embed_utterance(encoder_wav, return_partials=True)

        # # Add the utterance
        # name = speaker_name + "_gen_%05d" % np.random.randint(100000)
        # utterance = Utterance(name, speaker_name, wav, spec, embed, partial_embeds, True)
        # self.utterances.add(utterance)

    # Play some audio
    def play(self, wav, sample_rate):
        try:
            sd.stop()
            sd.play(wav, sample_rate)
        except Exception as e:
            print(e)
            print("Error in audio playback. Try selecting a different audio output device.")
            print("Your device must be connected before you start the toolbox.")

In [16]:
# Encoder
    
    
class HParams(object):
    def __init__(self, **kwargs): self.__dict__.update(kwargs)
    def __setitem__(self, key, value): setattr(self, key, value)
    def __getitem__(self, key): return getattr(self, key)
    def __repr__(self): return pprint.pformat(self.__dict__)

    def parse(self, string):
        # Overrides hparams from a comma-separated string of name=value pairs
        if len(string) > 0:
            overrides = [s.split("=") for s in string.split(",")]
            keys, values = zip(*overrides)
            keys = list(map(str.strip, keys))
            values = list(map(str.strip, values))
            for k in keys:
                self.__dict__[k] = ast.literal_eval(values[keys.index(k)])
        return self

hparams = HParams(
        ### Signal Processing (used in both synthesizer and vocoder)
        sample_rate = 16000,
        n_fft = 800,
        num_mels = 80,
        hop_size = 200,                             # Tacotron uses 12.5 ms frame shift (set to sample_rate * 0.0125)
        win_size = 800,                             # Tacotron uses 50 ms frame length (set to sample_rate * 0.050)
        fmin = 55,
        min_level_db = -100,
        ref_level_db = 20,
        max_abs_value = 4.,                         # Gradient explodes if too big, premature convergence if too small.
        preemphasis = 0.97,                         # Filter coefficient to use if preemphasize is True
        preemphasize = True,

        ### Tacotron Text-to-Speech (TTS)
        tts_embed_dims = 512,                       # Embedding dimension for the graphemes/phoneme inputs
        tts_encoder_dims = 256,
        tts_decoder_dims = 128,
        tts_postnet_dims = 512,
        tts_encoder_K = 5,
        tts_lstm_dims = 1024,
        tts_postnet_K = 5,
        tts_num_highways = 4,
        tts_dropout = 0.5,
        tts_cleaner_names = ["english_cleaners"],
        tts_stop_threshold = -3.4,                  # Value below which audio generation ends.
                                                    # For example, for a range of [-4, 4], this
                                                    # will terminate the sequence at the first
                                                    # frame that has all values < -3.4

        ### Tacotron Training
        tts_schedule = [(2,  1e-3,  20_000,  12),   # Progressive training schedule
                        (2,  5e-4,  40_000,  12),   # (r, lr, step, batch_size)
                        (2,  2e-4,  80_000,  12),   #
                        (2,  1e-4, 160_000,  12),   # r = reduction factor (# of mel frames
                        (2,  3e-5, 320_000,  12),   #     synthesized for each decoder iteration)
                        (2,  1e-5, 640_000,  12)],  # lr = learning rate

        tts_clip_grad_norm = 1.0,                   # clips the gradient norm to prevent explosion - set to None if not needed
        tts_eval_interval = 500,                    # Number of steps between model evaluation (sample generation)
                                                    # Set to -1 to generate after completing epoch, or 0 to disable

        tts_eval_num_samples = 1,                   # Makes this number of samples

        ### Data Preprocessing
        max_mel_frames = 900,
        rescale = True,
        rescaling_max = 0.9,
        synthesis_batch_size = 16,                  # For vocoder preprocessing and inference.

        ### Mel Visualization and Griffin-Lim
        signal_normalization = True,
        power = 1.5,
        griffin_lim_iters = 60,

        ### Audio processing options
        fmax = 7600,                                # Should not exceed (sample_rate // 2)
        allow_clipping_in_normalization = True,     # Used when signal_normalization = True
        clip_mels_length = True,                    # If true, discards samples exceeding max_mel_frames
        use_lws = False,                            # "Fast spectrogram phase recovery using local weighted sums"
        symmetric_mels = True,                      # Sets mel range to [-max_abs_value, max_abs_value] if True,
                                                    #               and [0, max_abs_value] if False
        trim_silence = True,                        # Use with sample_rate of 16000 for best results

        ### SV2TTS
        speaker_embedding_size = 256,               # Dimension for the speaker embedding
        silence_min_duration_split = 0.4,           # Duration in seconds of a silence for an utterance to be split
        utterance_min_duration = 1.6,               # Duration in seconds below which utterances are discarded
        )

def hparams_debug_string():
    return str(hparams)

_model = None
    
class audio:
    try:
        import webrtcvad
    except:
        warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")
        webrtcvad=None

    int16_max = (2 ** 15) - 1


    def preprocess_wav(fpath_or_wav: Union[str, Path, np.ndarray],
                       source_sr: Optional[int] = None,
                       normalize: Optional[bool] = True,
                       trim_silence: Optional[bool] = True):
        """
        Applies the preprocessing operations used in training the Speaker Encoder to a waveform 
        either on disk or in memory. The waveform will be resampled to match the data hyperparameters.

        :param fpath_or_wav: either a filepath to an audio file (many extensions are supported, not 
        just .wav), either the waveform as a numpy array of floats.
        :param source_sr: if passing an audio waveform, the sampling rate of the waveform before 
        preprocessing. After preprocessing, the waveform's sampling rate will match the data 
        hyperparameters. If passing a filepath, the sampling rate will be automatically detected and 
        this argument will be ignored.
        """
        # Load the wav from disk if needed
        if isinstance(fpath_or_wav, str) or isinstance(fpath_or_wav, Path):
            wav, source_sr = librosa.load(str(fpath_or_wav), sr=None)
        else:
            wav = fpath_or_wav

        # Resample the wav if needed
        if source_sr is not None and source_sr != sampling_rate:
            wav = librosa.resample(wav, source_sr, sampling_rate)

        # Apply the preprocessing: normalize volume and shorten long silences 
        if normalize:
            wav = normalize_volume(wav, audio_norm_target_dBFS, increase_only=True)
        if webrtcvad and trim_silence:
            wav = trim_long_silences(wav)

        return wav


    def wav_to_mel_spectrogram(wav):
        """
        Derives a mel spectrogram ready to be used by the encoder from a preprocessed audio waveform.
        Note: this not a log-mel spectrogram.
        """
        frames = librosa.feature.melspectrogram(
            wav,
            sampling_rate,
            n_fft=int(sampling_rate * mel_window_length / 1000),
            hop_length=int(sampling_rate * mel_window_step / 1000),
            n_mels=mel_n_channels
        )
        return frames.astype(np.float32).T


    def trim_long_silences(wav):
        """
        Ensures that segments without voice in the waveform remain no longer than a 
        threshold determined by the VAD parameters in params.py.

        :param wav: the raw waveform as a numpy array of floats 
        :return: the same waveform with silences trimmed away (length <= original wav length)
        """
        # Compute the voice detection window size
        samples_per_window = (vad_window_length * sampling_rate) // 1000

        # Trim the end of the audio to have a multiple of the window size
        wav = wav[:len(wav) - (len(wav) % samples_per_window)]

        # Convert the float waveform to 16-bit mono PCM
        pcm_wave = struct.pack("%dh" % len(wav), *(np.round(wav * int16_max)).astype(np.int16))

        # Perform voice activation detection
        voice_flags = []
        vad = webrtcvad.Vad(mode=3)
        for window_start in range(0, len(wav), samples_per_window):
            window_end = window_start + samples_per_window
            voice_flags.append(vad.is_speech(pcm_wave[window_start * 2:window_end * 2],
                                             sample_rate=sampling_rate))
        voice_flags = np.array(voice_flags)

        # Smooth the voice detection with a moving average
        def moving_average(array, width):
            array_padded = np.concatenate((np.zeros((width - 1) // 2), array, np.zeros(width // 2)))
            ret = np.cumsum(array_padded, dtype=float)
            ret[width:] = ret[width:] - ret[:-width]
            return ret[width - 1:] / width

        audio_mask = moving_average(voice_flags, vad_moving_average_width)
        audio_mask = np.round(audio_mask).astype(np.bool)

        # Dilate the voiced regions
        audio_mask = binary_dilation(audio_mask, np.ones(vad_max_silence_length + 1))
        audio_mask = np.repeat(audio_mask, samples_per_window)

        return wav[audio_mask == True]


    def normalize_volume(wav, target_dBFS, increase_only=False, decrease_only=False):
        if increase_only and decrease_only:
            raise ValueError("Both increase only and decrease only are set")
        dBFS_change = target_dBFS - 10 * np.log10(np.mean(wav ** 2))
        if (dBFS_change < 0 and increase_only) or (dBFS_change > 0 and decrease_only):
            return wav
        return wav * (10 ** (dBFS_change / 20))
    
    
class SpeakerEncoder(nn.Module):
    def __init__(self, device, loss_device):
        super().__init__()
        self.loss_device = loss_device

        # Network defition
        self.lstm = nn.LSTM(input_size=mel_n_channels,
                            hidden_size=model_hidden_size, 
                            num_layers=model_num_layers, 
                            batch_first=True).to(device)
        self.linear = nn.Linear(in_features=model_hidden_size, 
                                out_features=model_embedding_size).to(device)
        self.relu = torch.nn.ReLU().to(device)

        # Cosine similarity scaling (with fixed initial parameter values)
        self.similarity_weight = nn.Parameter(torch.tensor([10.])).to(loss_device)
        self.similarity_bias = nn.Parameter(torch.tensor([-5.])).to(loss_device)

        # Loss
        self.loss_fn = nn.CrossEntropyLoss().to(loss_device)

    def do_gradient_ops(self):
        # Gradient scale
        self.similarity_weight.grad *= 0.01
        self.similarity_bias.grad *= 0.01

        # Gradient clipping
        clip_grad_norm_(self.parameters(), 3, norm_type=2)

    def forward(self, utterances, hidden_init=None):
        """
        Computes the embeddings of a batch of utterance spectrograms.

        :param utterances: batch of mel-scale filterbanks of same duration as a tensor of shape 
        (batch_size, n_frames, n_channels) 
        :param hidden_init: initial hidden state of the LSTM as a tensor of shape (num_layers, 
        batch_size, hidden_size). Will default to a tensor of zeros if None.
        :return: the embeddings as a tensor of shape (batch_size, embedding_size)
        """
        # Pass the input through the LSTM layers and retrieve all outputs, the final hidden state
        # and the final cell state.
        out, (hidden, cell) = self.lstm(utterances, hidden_init)

        # We take only the hidden state of the last layer
        embeds_raw = self.relu(self.linear(hidden[-1]))

        # L2-normalize it
        embeds = embeds_raw / (torch.norm(embeds_raw, dim=1, keepdim=True) + 1e-5)        

        return embeds

    def similarity_matrix(self, embeds):
        """
        Computes the similarity matrix according the section 2.1 of GE2E.

        :param embeds: the embeddings as a tensor of shape (speakers_per_batch, 
        utterances_per_speaker, embedding_size)
        :return: the similarity matrix as a tensor of shape (speakers_per_batch,
        utterances_per_speaker, speakers_per_batch)
        """
        speakers_per_batch, utterances_per_speaker = embeds.shape[:2]

        # Inclusive centroids (1 per speaker). Cloning is needed for reverse differentiation
        centroids_incl = torch.mean(embeds, dim=1, keepdim=True)
        centroids_incl = centroids_incl.clone() / (torch.norm(centroids_incl, dim=2, keepdim=True) + 1e-5)

        # Exclusive centroids (1 per utterance)
        centroids_excl = (torch.sum(embeds, dim=1, keepdim=True) - embeds)
        centroids_excl /= (utterances_per_speaker - 1)
        centroids_excl = centroids_excl.clone() / (torch.norm(centroids_excl, dim=2, keepdim=True) + 1e-5)

        # Similarity matrix. The cosine similarity of already 2-normed vectors is simply the dot
        # product of these vectors (which is just an element-wise multiplication reduced by a sum).
        # We vectorize the computation for efficiency.
        sim_matrix = torch.zeros(speakers_per_batch, utterances_per_speaker,
                                 speakers_per_batch).to(self.loss_device)
        mask_matrix = 1 - np.eye(speakers_per_batch, dtype=np.int)
        for j in range(speakers_per_batch):
            mask = np.where(mask_matrix[j])[0]
            sim_matrix[mask, :, j] = (embeds[mask] * centroids_incl[j]).sum(dim=2)
            sim_matrix[j, :, j] = (embeds[j] * centroids_excl[j]).sum(dim=1)

        ## Even more vectorized version (slower maybe because of transpose)
        # sim_matrix2 = torch.zeros(speakers_per_batch, speakers_per_batch, utterances_per_speaker
        #                           ).to(self.loss_device)
        # eye = np.eye(speakers_per_batch, dtype=np.int)
        # mask = np.where(1 - eye)
        # sim_matrix2[mask] = (embeds[mask[0]] * centroids_incl[mask[1]]).sum(dim=2)
        # mask = np.where(eye)
        # sim_matrix2[mask] = (embeds * centroids_excl).sum(dim=2)
        # sim_matrix2 = sim_matrix2.transpose(1, 2)

        sim_matrix = sim_matrix * self.similarity_weight + self.similarity_bias
        return sim_matrix

    def loss(self, embeds):
        """
        Computes the softmax loss according the section 2.1 of GE2E.

        :param embeds: the embeddings as a tensor of shape (speakers_per_batch, 
        utterances_per_speaker, embedding_size)
        :return: the loss and the EER for this batch of embeddings.
        """
        speakers_per_batch, utterances_per_speaker = embeds.shape[:2]

        # Loss
        sim_matrix = self.similarity_matrix(embeds)
        sim_matrix = sim_matrix.reshape((speakers_per_batch * utterances_per_speaker, 
                                         speakers_per_batch))
        ground_truth = np.repeat(np.arange(speakers_per_batch), utterances_per_speaker)
        target = torch.from_numpy(ground_truth).long().to(self.loss_device)
        loss = self.loss_fn(sim_matrix, target)

        # EER (not backpropagated)
        with torch.no_grad():
            inv_argmax = lambda i: np.eye(1, speakers_per_batch, i, dtype=np.int)[0]
            labels = np.array([inv_argmax(i) for i in ground_truth])
            preds = sim_matrix.detach().cpu().numpy()

            # Snippet from https://yangcha.github.io/EER-ROC/
            fpr, tpr, thresholds = roc_curve(labels.flatten(), preds.flatten())           
            eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)

        return loss, eer

_model = None # type: SpeakerEncoder
_device = None # type: torch.device
    
    
    
class encoder:
    ## Mel-filterbank
    mel_window_length = 25  # In milliseconds
    mel_window_step = 10    # In milliseconds
    mel_n_channels = 40


    ## Audio
    sampling_rate = 16000
    # Number of spectrogram frames in a partial utterance
    partials_n_frames = 160     # 1600 ms
    # Number of spectrogram frames at inference
    inference_n_frames = 80     #  800 ms


    ## Voice Activation Detection
    # Window size of the VAD. Must be either 10, 20 or 30 milliseconds.
    # This sets the granularity of the VAD. Should not need to be changed.
    vad_window_length = 30  # In milliseconds
    # Number of frames to average together when performing the moving average smoothing.
    # The larger this value, the larger the VAD variations must be to not get smoothed out. 
    vad_moving_average_width = 8
    # Maximum number of consecutive silent frames a segment can have.
    vad_max_silence_length = 6


    ## Audio volume normalization
    audio_norm_target_dBFS = -30


    ## Model parameters
    model_hidden_size = 256
    model_embedding_size = 256
    model_num_layers = 3


    ## Training parameters
    learning_rate_init = 1e-4
    speakers_per_batch = 64
    utterances_per_speaker = 10


    class SpeakerEncoder(nn.Module):
        def __init__(self, device, loss_device):
            super().__init__()
            self.loss_device = loss_device

            # Network defition
            self.lstm = nn.LSTM(input_size=mel_n_channels,
                                hidden_size=model_hidden_size, 
                                num_layers=model_num_layers, 
                                batch_first=True).to(device)
            self.linear = nn.Linear(in_features=model_hidden_size, 
                                    out_features=model_embedding_size).to(device)
            self.relu = torch.nn.ReLU().to(device)

            # Cosine similarity scaling (with fixed initial parameter values)
            self.similarity_weight = nn.Parameter(torch.tensor([10.])).to(loss_device)
            self.similarity_bias = nn.Parameter(torch.tensor([-5.])).to(loss_device)

            # Loss
            self.loss_fn = nn.CrossEntropyLoss().to(loss_device)

        def do_gradient_ops(self):
            # Gradient scale
            self.similarity_weight.grad *= 0.01
            self.similarity_bias.grad *= 0.01

            # Gradient clipping
            clip_grad_norm_(self.parameters(), 3, norm_type=2)

        def forward(self, utterances, hidden_init=None):
            """
            Computes the embeddings of a batch of utterance spectrograms.

            :param utterances: batch of mel-scale filterbanks of same duration as a tensor of shape 
            (batch_size, n_frames, n_channels) 
            :param hidden_init: initial hidden state of the LSTM as a tensor of shape (num_layers, 
            batch_size, hidden_size). Will default to a tensor of zeros if None.
            :return: the embeddings as a tensor of shape (batch_size, embedding_size)
            """
            # Pass the input through the LSTM layers and retrieve all outputs, the final hidden state
            # and the final cell state.
            out, (hidden, cell) = self.lstm(utterances, hidden_init)

            # We take only the hidden state of the last layer
            embeds_raw = self.relu(self.linear(hidden[-1]))

            # L2-normalize it
            embeds = embeds_raw / (torch.norm(embeds_raw, dim=1, keepdim=True) + 1e-5)        

            return embeds

        def similarity_matrix(self, embeds):
            """
            Computes the similarity matrix according the section 2.1 of GE2E.

            :param embeds: the embeddings as a tensor of shape (speakers_per_batch, 
            utterances_per_speaker, embedding_size)
            :return: the similarity matrix as a tensor of shape (speakers_per_batch,
            utterances_per_speaker, speakers_per_batch)
            """
            speakers_per_batch, utterances_per_speaker = embeds.shape[:2]

            # Inclusive centroids (1 per speaker). Cloning is needed for reverse differentiation
            centroids_incl = torch.mean(embeds, dim=1, keepdim=True)
            centroids_incl = centroids_incl.clone() / (torch.norm(centroids_incl, dim=2, keepdim=True) + 1e-5)

            # Exclusive centroids (1 per utterance)
            centroids_excl = (torch.sum(embeds, dim=1, keepdim=True) - embeds)
            centroids_excl /= (utterances_per_speaker - 1)
            centroids_excl = centroids_excl.clone() / (torch.norm(centroids_excl, dim=2, keepdim=True) + 1e-5)

            # Similarity matrix. The cosine similarity of already 2-normed vectors is simply the dot
            # product of these vectors (which is just an element-wise multiplication reduced by a sum).
            # We vectorize the computation for efficiency.
            sim_matrix = torch.zeros(speakers_per_batch, utterances_per_speaker,
                                     speakers_per_batch).to(self.loss_device)
            mask_matrix = 1 - np.eye(speakers_per_batch, dtype=np.int)
            for j in range(speakers_per_batch):
                mask = np.where(mask_matrix[j])[0]
                sim_matrix[mask, :, j] = (embeds[mask] * centroids_incl[j]).sum(dim=2)
                sim_matrix[j, :, j] = (embeds[j] * centroids_excl[j]).sum(dim=1)

            ## Even more vectorized version (slower maybe because of transpose)
            # sim_matrix2 = torch.zeros(speakers_per_batch, speakers_per_batch, utterances_per_speaker
            #                           ).to(self.loss_device)
            # eye = np.eye(speakers_per_batch, dtype=np.int)
            # mask = np.where(1 - eye)
            # sim_matrix2[mask] = (embeds[mask[0]] * centroids_incl[mask[1]]).sum(dim=2)
            # mask = np.where(eye)
            # sim_matrix2[mask] = (embeds * centroids_excl).sum(dim=2)
            # sim_matrix2 = sim_matrix2.transpose(1, 2)

            sim_matrix = sim_matrix * self.similarity_weight + self.similarity_bias
            return sim_matrix

        def loss(self, embeds):
            """
            Computes the softmax loss according the section 2.1 of GE2E.

            :param embeds: the embeddings as a tensor of shape (speakers_per_batch, 
            utterances_per_speaker, embedding_size)
            :return: the loss and the EER for this batch of embeddings.
            """
            speakers_per_batch, utterances_per_speaker = embeds.shape[:2]

            # Loss
            sim_matrix = self.similarity_matrix(embeds)
            sim_matrix = sim_matrix.reshape((speakers_per_batch * utterances_per_speaker, 
                                             speakers_per_batch))
            ground_truth = np.repeat(np.arange(speakers_per_batch), utterances_per_speaker)
            target = torch.from_numpy(ground_truth).long().to(self.loss_device)
            loss = self.loss_fn(sim_matrix, target)

            # EER (not backpropagated)
            with torch.no_grad():
                inv_argmax = lambda i: np.eye(1, speakers_per_batch, i, dtype=np.int)[0]
                labels = np.array([inv_argmax(i) for i in ground_truth])
                preds = sim_matrix.detach().cpu().numpy()

                # Snippet from https://yangcha.github.io/EER-ROC/
                fpr, tpr, thresholds = roc_curve(labels.flatten(), preds.flatten())           
                eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)

            return loss, eer
    
    _model = None # type: SpeakerEncoder
    _device = None # type: torch.device


    def load_model(weights_fpath: Path, device=None):
        """
        Loads the model in memory. If this function is not explicitely called, it will be run on the
        first call to embed_frames() with the default weights file.

        :param weights_fpath: the path to saved model weights.
        :param device: either a torch device or the name of a torch device (e.g. "cpu", "cuda"). The
        model will be loaded and will run on this device. Outputs will however always be on the cpu.
        If None, will default to your GPU if it"s available, otherwise your CPU.
        """
        # TODO: I think the slow loading of the encoder might have something to do with the device it
        #   was saved on. Worth investigating.
        global _model, _device
        if device is None:
            _device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        elif isinstance(device, str):
            _device = torch.device(device)
        _model = SpeakerEncoder(_device, torch.device("cpu"))
        try:
            checkpoint = torch.load(weights_fpath, _device)
        except AttributeError:
            checkpoint = torch.load(weights_fpath.name, _device)
        _model.load_state_dict(checkpoint["model_state"])
        _model.eval()
        print("Loaded encoder \"%s\" trained to step %d" % (weights_fpath.name, checkpoint["step"]))


    def is_loaded():
        return _model is not None


    def embed_frames_batch(frames_batch):
        """
        Computes embeddings for a batch of mel spectrogram.

        :param frames_batch: a batch mel of spectrogram as a numpy array of float32 of shape
        (batch_size, n_frames, n_channels)
        :return: the embeddings as a numpy array of float32 of shape (batch_size, model_embedding_size)
        """
        if _model is None:
            raise Exception("Model was not loaded. Call load_model() before inference.")

        frames = torch.from_numpy(frames_batch).to(_device)
        embed = _model.forward(frames).detach().cpu().numpy()
        return embed


    def compute_partial_slices(n_samples, partial_utterance_n_frames=partials_n_frames,
                               min_pad_coverage=0.75, overlap=0.5):
        """
        Computes where to split an utterance waveform and its corresponding mel spectrogram to obtain
        partial utterances of <partial_utterance_n_frames> each. Both the waveform and the mel
        spectrogram slices are returned, so as to make each partial utterance waveform correspond to
        its spectrogram. This function assumes that the mel spectrogram parameters used are those
        defined in params_data.py.

        The returned ranges may be indexing further than the length of the waveform. It is
        recommended that you pad the waveform with zeros up to wave_slices[-1].stop.

        :param n_samples: the number of samples in the waveform
        :param partial_utterance_n_frames: the number of mel spectrogram frames in each partial
        utterance
        :param min_pad_coverage: when reaching the last partial utterance, it may or may not have
        enough frames. If at least <min_pad_coverage> of <partial_utterance_n_frames> are present,
        then the last partial utterance will be considered, as if we padded the audio. Otherwise,
        it will be discarded, as if we trimmed the audio. If there aren't enough frames for 1 partial
        utterance, this parameter is ignored so that the function always returns at least 1 slice.
        :param overlap: by how much the partial utterance should overlap. If set to 0, the partial
        utterances are entirely disjoint.
        :return: the waveform slices and mel spectrogram slices as lists of array slices. Index
        respectively the waveform and the mel spectrogram with these slices to obtain the partial
        utterances.
        """
        assert 0 <= overlap < 1
        assert 0 < min_pad_coverage <= 1

        samples_per_frame = int((sampling_rate * mel_window_step / 1000))
        n_frames = int(np.ceil((n_samples + 1) / samples_per_frame))
        frame_step = max(int(np.round(partial_utterance_n_frames * (1 - overlap))), 1)

        # Compute the slices
        wav_slices, mel_slices = [], []
        steps = max(1, n_frames - partial_utterance_n_frames + frame_step + 1)
        for i in range(0, steps, frame_step):
            mel_range = np.array([i, i + partial_utterance_n_frames])
            wav_range = mel_range * samples_per_frame
            mel_slices.append(slice(*mel_range))
            wav_slices.append(slice(*wav_range))

        # Evaluate whether extra padding is warranted or not
        last_wav_range = wav_slices[-1]
        coverage = (n_samples - last_wav_range.start) / (last_wav_range.stop - last_wav_range.start)
        if coverage < min_pad_coverage and len(mel_slices) > 1:
            mel_slices = mel_slices[:-1]
            wav_slices = wav_slices[:-1]

        return wav_slices, mel_slices


    def embed_utterance(wav, using_partials=True, return_partials=False, **kwargs):
        """
        Computes an embedding for a single utterance.

        # TODO: handle multiple wavs to benefit from batching on GPU
        :param wav: a preprocessed (see audio.py) utterance waveform as a numpy array of float32
        :param using_partials: if True, then the utterance is split in partial utterances of
        <partial_utterance_n_frames> frames and the utterance embedding is computed from their
        normalized average. If False, the utterance is instead computed from feeding the entire
        spectogram to the network.
        :param return_partials: if True, the partial embeddings will also be returned along with the
        wav slices that correspond to the partial embeddings.
        :param kwargs: additional arguments to compute_partial_splits()
        :return: the embedding as a numpy array of float32 of shape (model_embedding_size,). If
        <return_partials> is True, the partial utterances as a numpy array of float32 of shape
        (n_partials, model_embedding_size) and the wav partials as a list of slices will also be
        returned. If <using_partials> is simultaneously set to False, both these values will be None
        instead.
        """
        # Process the entire utterance if not using partials
        if not using_partials:
            frames = audio.wav_to_mel_spectrogram(wav)
            embed = embed_frames_batch(frames[None, ...])[0]
            if return_partials:
                return embed, None, None
            return embed

        # Compute where to split the utterance into partials and pad if necessary
        wave_slices, mel_slices = compute_partial_slices(len(wav), **kwargs)
        max_wave_length = wave_slices[-1].stop
        if max_wave_length >= len(wav):
            wav = np.pad(wav, (0, max_wave_length - len(wav)), "constant")

        # Split the utterance into partials
        frames = audio.wav_to_mel_spectrogram(wav)
        frames_batch = np.array([frames[s] for s in mel_slices])
        partial_embeds = embed_frames_batch(frames_batch)

        # Compute the utterance embedding from the partial embeddings
        raw_embed = np.mean(partial_embeds, axis=0)
        embed = raw_embed / np.linalg.norm(raw_embed, 2)

        if return_partials:
            return embed, partial_embeds, wave_slices
        return embed


    def embed_speaker(wavs, **kwargs):
        raise NotImplemented()


    def plot_embedding_as_heatmap(embed, ax=None, title="", shape=None, color_range=(0, 0.30)):
        import matplotlib.pyplot as plt
        if ax is None:
            ax = plt.gca()

        if shape is None:
            height = int(np.sqrt(len(embed)))
            shape = (height, -1)
        embed = embed.reshape(shape)

        cmap = cm.get_cmap()
        mappable = ax.imshow(embed, cmap=cmap)
        cbar = plt.colorbar(mappable, ax=ax, fraction=0.046, pad=0.04)
        sm = cm.ScalarMappable(cmap=cmap)
        sm.set_clim(*color_range)

        ax.set_xticks([]), ax.set_yticks([])
        ax.set_title(title)
    
    int16_max = (2 ** 15) - 1


    def preprocess_wav(fpath_or_wav: Union[str, Path, np.ndarray],
                       source_sr: Optional[int] = None,
                       normalize: Optional[bool] = True,
                       trim_silence: Optional[bool] = True):
        """
        Applies the preprocessing operations used in training the Speaker Encoder to a waveform 
        either on disk or in memory. The waveform will be resampled to match the data hyperparameters.

        :param fpath_or_wav: either a filepath to an audio file (many extensions are supported, not 
        just .wav), either the waveform as a numpy array of floats.
        :param source_sr: if passing an audio waveform, the sampling rate of the waveform before 
        preprocessing. After preprocessing, the waveform's sampling rate will match the data 
        hyperparameters. If passing a filepath, the sampling rate will be automatically detected and 
        this argument will be ignored.
        """
        # Load the wav from disk if needed
        if isinstance(fpath_or_wav, str) or isinstance(fpath_or_wav, Path):
            wav, source_sr = librosa.load(str(fpath_or_wav), sr=None)
        else:
            wav = fpath_or_wav

        # Resample the wav if needed
        if source_sr is not None and source_sr != sampling_rate:
            wav = librosa.resample(wav, source_sr, sampling_rate)

        # Apply the preprocessing: normalize volume and shorten long silences 
        if normalize:
            wav = normalize_volume(wav, audio_norm_target_dBFS, increase_only=True)
        if webrtcvad and trim_silence:
            wav = trim_long_silences(wav)

        return wav


    def wav_to_mel_spectrogram(wav):
        """
        Derives a mel spectrogram ready to be used by the encoder from a preprocessed audio waveform.
        Note: this not a log-mel spectrogram.
        """
        frames = librosa.feature.melspectrogram(
            wav,
            sampling_rate,
            n_fft=int(sampling_rate * mel_window_length / 1000),
            hop_length=int(sampling_rate * mel_window_step / 1000),
            n_mels=mel_n_channels
        )
        return frames.astype(np.float32).T


    def trim_long_silences(wav):
        """
        Ensures that segments without voice in the waveform remain no longer than a 
        threshold determined by the VAD parameters in params.py.

        :param wav: the raw waveform as a numpy array of floats 
        :return: the same waveform with silences trimmed away (length <= original wav length)
        """
        # Compute the voice detection window size
        samples_per_window = (vad_window_length * sampling_rate) // 1000

        # Trim the end of the audio to have a multiple of the window size
        wav = wav[:len(wav) - (len(wav) % samples_per_window)]

        # Convert the float waveform to 16-bit mono PCM
        pcm_wave = struct.pack("%dh" % len(wav), *(np.round(wav * int16_max)).astype(np.int16))

        # Perform voice activation detection
        voice_flags = []
        vad = webrtcvad.Vad(mode=3)
        for window_start in range(0, len(wav), samples_per_window):
            window_end = window_start + samples_per_window
            voice_flags.append(vad.is_speech(pcm_wave[window_start * 2:window_end * 2],
                                             sample_rate=sampling_rate))
        voice_flags = np.array(voice_flags)

        # Smooth the voice detection with a moving average
        def moving_average(array, width):
            array_padded = np.concatenate((np.zeros((width - 1) // 2), array, np.zeros(width // 2)))
            ret = np.cumsum(array_padded, dtype=float)
            ret[width:] = ret[width:] - ret[:-width]
            return ret[width - 1:] / width

        audio_mask = moving_average(voice_flags, vad_moving_average_width)
        audio_mask = np.round(audio_mask).astype(np.bool)

        # Dilate the voiced regions
        audio_mask = binary_dilation(audio_mask, np.ones(vad_max_silence_length + 1))
        audio_mask = np.repeat(audio_mask, samples_per_window)

        return wav[audio_mask == True]


    def normalize_volume(wav, target_dBFS, increase_only=False, decrease_only=False):
        if increase_only and decrease_only:
            raise ValueError("Both increase only and decrease only are set")
        dBFS_change = target_dBFS - 10 * np.log10(np.mean(wav ** 2))
        if (dBFS_change < 0 and increase_only) or (dBFS_change > 0 and decrease_only):
            return wav
        return wav * (10 ** (dBFS_change / 20))


In [5]:
# Synthesizer
class Synthesizer:
    class audio:
        def load_wav(path, sr):
            return librosa.core.load(path, sr=sr)[0]

        def save_wav(wav, path, sr):
            wav *= 32767 / max(0.01, np.max(np.abs(wav)))
            #proposed by @dsmiller
            wavfile.write(path, sr, wav.astype(np.int16))

        def save_wavenet_wav(wav, path, sr):
            sf.write(path, wav.astype(np.float32), sr)

        def preemphasis(wav, k, preemphasize=True):
            if preemphasize:
                return signal.lfilter([1, -k], [1], wav)
            return wav

        def inv_preemphasis(wav, k, inv_preemphasize=True):
            if inv_preemphasize:
                return signal.lfilter([1], [1, -k], wav)
            return wav

        #From https://github.com/r9y9/wavenet_vocoder/blob/master/audio.py
        def start_and_end_indices(quantized, silence_threshold=2):
            for start in range(quantized.size):
                if abs(quantized[start] - 127) > silence_threshold:
                    break
            for end in range(quantized.size - 1, 1, -1):
                if abs(quantized[end] - 127) > silence_threshold:
                    break
            
            assert abs(quantized[start] - 127) > silence_threshold
            assert abs(quantized[end] - 127) > silence_threshold
            
            return start, end

        def get_hop_size(hparams):
            hop_size = hparams.hop_size
            if hop_size is None:
                assert hparams.frame_shift_ms is not None
                hop_size = int(hparams.frame_shift_ms / 1000 * hparams.sample_rate)
            return hop_size

        def linearspectrogram(wav, hparams):
            D = _stft(preemphasis(wav, hparams.preemphasis, hparams.preemphasize), hparams)
            S = _amp_to_db(np.abs(D), hparams) - hparams.ref_level_db
            
            if hparams.signal_normalization:
                return _normalize(S, hparams)
            return S

        def melspectrogram(wav, hparams):
            D = _stft(preemphasis(wav, hparams.preemphasis, hparams.preemphasize), hparams)
            S = _amp_to_db(_linear_to_mel(np.abs(D), hparams), hparams) - hparams.ref_level_db
            
            if hparams.signal_normalization:
                return _normalize(S, hparams)
            return S

        def inv_linear_spectrogram(linear_spectrogram, hparams):
            """Converts linear spectrogram to waveform using librosa"""
            if hparams.signal_normalization:
                D = _denormalize(linear_spectrogram, hparams)
            else:
                D = linear_spectrogram
            
            S = _db_to_amp(D + hparams.ref_level_db) #Convert back to linear
            
            if hparams.use_lws:
                processor = _lws_processor(hparams)
                D = processor.run_lws(S.astype(np.float64).T ** hparams.power)
                y = processor.istft(D).astype(np.float32)
                return inv_preemphasis(y, hparams.preemphasis, hparams.preemphasize)
            else:
                return inv_preemphasis(_griffin_lim(S ** hparams.power, hparams), hparams.preemphasis, hparams.preemphasize)

        def inv_mel_spectrogram(mel_spectrogram, hparams):
            """Converts mel spectrogram to waveform using librosa"""
            if hparams.signal_normalization:
                D = _denormalize(mel_spectrogram, hparams)
            else:
                D = mel_spectrogram
            
            S = _mel_to_linear(_db_to_amp(D + hparams.ref_level_db), hparams)  # Convert back to linear
            
            if hparams.use_lws:
                processor = _lws_processor(hparams)
                D = processor.run_lws(S.astype(np.float64).T ** hparams.power)
                y = processor.istft(D).astype(np.float32)
                return inv_preemphasis(y, hparams.preemphasis, hparams.preemphasize)
            else:
                return inv_preemphasis(_griffin_lim(S ** hparams.power, hparams), hparams.preemphasis, hparams.preemphasize)

        def _lws_processor(hparams):
            import lws
            return lws.lws(hparams.n_fft, get_hop_size(hparams), fftsize=hparams.win_size, mode="speech")

        def _griffin_lim(S, hparams):
            """librosa implementation of Griffin-Lim
            Based on https://github.com/librosa/librosa/issues/434
            """
            angles = np.exp(2j * np.pi * np.random.rand(*S.shape))
            S_complex = np.abs(S).astype(np.complex)
            y = _istft(S_complex * angles, hparams)
            for i in range(hparams.griffin_lim_iters):
                angles = np.exp(1j * np.angle(_stft(y, hparams)))
                y = _istft(S_complex * angles, hparams)
            return y

        def _stft(y, hparams):
            if hparams.use_lws:
                return _lws_processor(hparams).stft(y).T
            else:
                return librosa.stft(y=y, n_fft=hparams.n_fft, hop_length=get_hop_size(hparams), win_length=hparams.win_size)

        def _istft(y, hparams):
            return librosa.istft(y, hop_length=get_hop_size(hparams), win_length=hparams.win_size)

        ##########################################################
        #Those are only correct when using lws!!! (This was messing with Wavenet quality for a long time!)
        def num_frames(length, fsize, fshift):
            """Compute number of time frames of spectrogram
            """
            pad = (fsize - fshift)
            if length % fshift == 0:
                M = (length + pad * 2 - fsize) // fshift + 1
            else:
                M = (length + pad * 2 - fsize) // fshift + 2
            return M


        def pad_lr(x, fsize, fshift):
            """Compute left and right padding
            """
            M = num_frames(len(x), fsize, fshift)
            pad = (fsize - fshift)
            T = len(x) + 2 * pad
            r = (M - 1) * fshift + fsize - T
            return pad, pad + r
        ##########################################################
        #Librosa correct padding
        def librosa_pad_lr(x, fsize, fshift):
            return 0, (x.shape[0] // fshift + 1) * fshift - x.shape[0]

        # Conversions
        _mel_basis = None
        _inv_mel_basis = None

        def _linear_to_mel(spectogram, hparams):
            global _mel_basis
            if _mel_basis is None:
                _mel_basis = _build_mel_basis(hparams)
            return np.dot(_mel_basis, spectogram)

        def _mel_to_linear(mel_spectrogram, hparams):
            global _inv_mel_basis
            if _inv_mel_basis is None:
                _inv_mel_basis = np.linalg.pinv(_build_mel_basis(hparams))
            return np.maximum(1e-10, np.dot(_inv_mel_basis, mel_spectrogram))

        def _build_mel_basis(hparams):
            assert hparams.fmax <= hparams.sample_rate // 2
            return librosa.filters.mel(hparams.sample_rate, hparams.n_fft, n_mels=hparams.num_mels,
                                    fmin=hparams.fmin, fmax=hparams.fmax)

        def _amp_to_db(x, hparams):
            min_level = np.exp(hparams.min_level_db / 20 * np.log(10))
            return 20 * np.log10(np.maximum(min_level, x))

        def _db_to_amp(x):
            return np.power(10.0, (x) * 0.05)

        def _normalize(S, hparams):
            if hparams.allow_clipping_in_normalization:
                if hparams.symmetric_mels:
                    return np.clip((2 * hparams.max_abs_value) * ((S - hparams.min_level_db) / (-hparams.min_level_db)) - hparams.max_abs_value,
                                -hparams.max_abs_value, hparams.max_abs_value)
                else:
                    return np.clip(hparams.max_abs_value * ((S - hparams.min_level_db) / (-hparams.min_level_db)), 0, hparams.max_abs_value)
            
            assert S.max() <= 0 and S.min() - hparams.min_level_db >= 0
            if hparams.symmetric_mels:
                return (2 * hparams.max_abs_value) * ((S - hparams.min_level_db) / (-hparams.min_level_db)) - hparams.max_abs_value
            else:
                return hparams.max_abs_value * ((S - hparams.min_level_db) / (-hparams.min_level_db))

        def _denormalize(D, hparams):
            if hparams.allow_clipping_in_normalization:
                if hparams.symmetric_mels:
                    return (((np.clip(D, -hparams.max_abs_value,
                                    hparams.max_abs_value) + hparams.max_abs_value) * -hparams.min_level_db / (2 * hparams.max_abs_value))
                            + hparams.min_level_db)
                else:
                    return ((np.clip(D, 0, hparams.max_abs_value) * -hparams.min_level_db / hparams.max_abs_value) + hparams.min_level_db)
            
            if hparams.symmetric_mels:
                return (((D + hparams.max_abs_value) * -hparams.min_level_db / (2 * hparams.max_abs_value)) + hparams.min_level_db)
            else:
                return ((D * -hparams.min_level_db / hparams.max_abs_value) + hparams.min_level_db)

    class HParams(object):
        def __init__(self, **kwargs): self.__dict__.update(kwargs)
        def __setitem__(self, key, value): setattr(self, key, value)
        def __getitem__(self, key): return getattr(self, key)
        def __repr__(self): return pprint.pformat(self.__dict__)

        def parse(self, string):
            # Overrides hparams from a comma-separated string of name=value pairs
            if len(string) > 0:
                overrides = [s.split("=") for s in string.split(",")]
                keys, values = zip(*overrides)
                keys = list(map(str.strip, keys))
                values = list(map(str.strip, values))
                for k in keys:
                    self.__dict__[k] = ast.literal_eval(values[keys.index(k)])
            return self

    hparams = HParams(
            ### Signal Processing (used in both synthesizer and vocoder)
            sample_rate = 16000,
            n_fft = 800,
            num_mels = 80,
            hop_size = 200,                             # Tacotron uses 12.5 ms frame shift (set to sample_rate * 0.0125)
            win_size = 800,                             # Tacotron uses 50 ms frame length (set to sample_rate * 0.050)
            fmin = 55,
            min_level_db = -100,
            ref_level_db = 20,
            max_abs_value = 4.,                         # Gradient explodes if too big, premature convergence if too small.
            preemphasis = 0.97,                         # Filter coefficient to use if preemphasize is True
            preemphasize = True,

            ### Tacotron Text-to-Speech (TTS)
            tts_embed_dims = 512,                       # Embedding dimension for the graphemes/phoneme inputs
            tts_encoder_dims = 256,
            tts_decoder_dims = 128,
            tts_postnet_dims = 512,
            tts_encoder_K = 5,
            tts_lstm_dims = 1024,
            tts_postnet_K = 5,
            tts_num_highways = 4,
            tts_dropout = 0.5,
            tts_cleaner_names = ["english_cleaners"],
            tts_stop_threshold = -3.4,                  # Value below which audio generation ends.
                                                        # For example, for a range of [-4, 4], this
                                                        # will terminate the sequence at the first
                                                        # frame that has all values < -3.4

            ### Tacotron Training
            tts_schedule = [(2,  1e-3,  20_000,  12),   # Progressive training schedule
                            (2,  5e-4,  40_000,  12),   # (r, lr, step, batch_size)
                            (2,  2e-4,  80_000,  12),   #
                            (2,  1e-4, 160_000,  12),   # r = reduction factor (# of mel frames
                            (2,  3e-5, 320_000,  12),   #     synthesized for each decoder iteration)
                            (2,  1e-5, 640_000,  12)],  # lr = learning rate

            tts_clip_grad_norm = 1.0,                   # clips the gradient norm to prevent explosion - set to None if not needed
            tts_eval_interval = 500,                    # Number of steps between model evaluation (sample generation)
                                                        # Set to -1 to generate after completing epoch, or 0 to disable

            tts_eval_num_samples = 1,                   # Makes this number of samples

            ### Data Preprocessing
            max_mel_frames = 900,
            rescale = True,
            rescaling_max = 0.9,
            synthesis_batch_size = 16,                  # For vocoder preprocessing and inference.

            ### Mel Visualization and Griffin-Lim
            signal_normalization = True,
            power = 1.5,
            griffin_lim_iters = 60,

            ### Audio processing options
            fmax = 7600,                                # Should not exceed (sample_rate // 2)
            allow_clipping_in_normalization = True,     # Used when signal_normalization = True
            clip_mels_length = True,                    # If true, discards samples exceeding max_mel_frames
            use_lws = False,                            # "Fast spectrogram phase recovery using local weighted sums"
            symmetric_mels = True,                      # Sets mel range to [-max_abs_value, max_abs_value] if True,
                                                        #               and [0, max_abs_value] if False
            trim_silence = True,                        # Use with sample_rate of 16000 for best results

            ### SV2TTS
            speaker_embedding_size = 256,               # Dimension for the speaker embedding
            silence_min_duration_split = 0.4,           # Duration in seconds of a silence for an utterance to be split
            utterance_min_duration = 1.6,               # Duration in seconds below which utterances are discarded
            )

    def hparams_debug_string():
        return str(hparams)
    
    
    _pad        = "_"
    _eos        = "~"
    _characters = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz!\'\"(),-.:;? "
    symbols = [_pad, _eos] + list(_characters) #+ _arpabet
    
    class cleaners:
        # Regular expression matching whitespace:
        _whitespace_re = re.compile(r"\s+")

        # List of (regular expression, replacement) pairs for abbreviations:
        _abbreviations = [(re.compile("\\b%s\\." % x[0], re.IGNORECASE), x[1]) for x in [
            ("mrs", "misess"),
            ("mr", "mister"),
            ("dr", "doctor"),
            ("st", "saint"),
            ("co", "company"),
            ("jr", "junior"),
            ("maj", "major"),
            ("gen", "general"),
            ("drs", "doctors"),
            ("rev", "reverend"),
            ("lt", "lieutenant"),
            ("hon", "honorable"),
            ("sgt", "sergeant"),
            ("capt", "captain"),
            ("esq", "esquire"),
            ("ltd", "limited"),
            ("col", "colonel"),
            ("ft", "fort"),
        ]]


        def expand_abbreviations(text):
            for regex, replacement in _abbreviations:
                text = re.sub(regex, replacement, text)
            return text


        def expand_numbers(text):
            return normalize_numbers(text)


        def lowercase(text):
            """lowercase input tokens."""
            return text.lower()


        def collapse_whitespace(text):
            return re.sub(_whitespace_re, " ", text)


        def convert_to_ascii(text):
            return unidecode(text)


        def basic_cleaners(text):
            """Basic pipeline that lowercases and collapses whitespace without transliteration."""
            text = lowercase(text)
            text = collapse_whitespace(text)
            return text


        def transliteration_cleaners(text):
            """Pipeline for non-English text that transliterates to ASCII."""
            text = convert_to_ascii(text)
            text = lowercase(text)
            text = collapse_whitespace(text)
            return text


        def english_cleaners(text):
            """Pipeline for English text, including number and abbreviation expansion."""
            text = convert_to_ascii(text)
            text = lowercase(text)
            text = expand_numbers(text)
            text = expand_abbreviations(text)
            text = collapse_whitespace(text)
            return text

    
    
    class Tacotron(nn.Module):
        def __init__(self, embed_dims, num_chars, encoder_dims, decoder_dims, n_mels, 
                     fft_bins, postnet_dims, encoder_K, lstm_dims, postnet_K, num_highways,
                     dropout, stop_threshold, speaker_embedding_size):
            super().__init__()
            self.n_mels = n_mels
            self.lstm_dims = lstm_dims
            self.encoder_dims = encoder_dims
            self.decoder_dims = decoder_dims
            self.speaker_embedding_size = speaker_embedding_size
            self.encoder = Encoder(embed_dims, num_chars, encoder_dims,
                                   encoder_K, num_highways, dropout)
            self.encoder_proj = nn.Linear(encoder_dims + speaker_embedding_size, decoder_dims, bias=False)
            self.decoder = Decoder(n_mels, encoder_dims, decoder_dims, lstm_dims,
                                   dropout, speaker_embedding_size)
            self.postnet = CBHG(postnet_K, n_mels, postnet_dims,
                                [postnet_dims, fft_bins], num_highways)
            self.post_proj = nn.Linear(postnet_dims, fft_bins, bias=False)

            self.init_model()
            self.num_params()

            self.register_buffer("step", torch.zeros(1, dtype=torch.long))
            self.register_buffer("stop_threshold", torch.tensor(stop_threshold, dtype=torch.float32))

        @property
        def r(self):
            return self.decoder.r.item()

        @r.setter
        def r(self, value):
            self.decoder.r = self.decoder.r.new_tensor(value, requires_grad=False)

        def forward(self, x, m, speaker_embedding):
            device = next(self.parameters()).device  # use same device as parameters

            self.step += 1
            batch_size, _, steps  = m.size()

            # Initialise all hidden states and pack into tuple
            attn_hidden = torch.zeros(batch_size, self.decoder_dims, device=device)
            rnn1_hidden = torch.zeros(batch_size, self.lstm_dims, device=device)
            rnn2_hidden = torch.zeros(batch_size, self.lstm_dims, device=device)
            hidden_states = (attn_hidden, rnn1_hidden, rnn2_hidden)

            # Initialise all lstm cell states and pack into tuple
            rnn1_cell = torch.zeros(batch_size, self.lstm_dims, device=device)
            rnn2_cell = torch.zeros(batch_size, self.lstm_dims, device=device)
            cell_states = (rnn1_cell, rnn2_cell)

            # <GO> Frame for start of decoder loop
            go_frame = torch.zeros(batch_size, self.n_mels, device=device)

            # Need an initial context vector
            context_vec = torch.zeros(batch_size, self.encoder_dims + self.speaker_embedding_size, device=device)

            # SV2TTS: Run the encoder with the speaker embedding
            # The projection avoids unnecessary matmuls in the decoder loop
            encoder_seq = self.encoder(x, speaker_embedding)
            encoder_seq_proj = self.encoder_proj(encoder_seq)

            # Need a couple of lists for outputs
            mel_outputs, attn_scores, stop_outputs = [], [], []

            # Run the decoder loop
            for t in range(0, steps, self.r):
                prenet_in = m[:, :, t - 1] if t > 0 else go_frame
                mel_frames, scores, hidden_states, cell_states, context_vec, stop_tokens = \
                    self.decoder(encoder_seq, encoder_seq_proj, prenet_in,
                                 hidden_states, cell_states, context_vec, t, x)
                mel_outputs.append(mel_frames)
                attn_scores.append(scores)
                stop_outputs.extend([stop_tokens] * self.r)

            # Concat the mel outputs into sequence
            mel_outputs = torch.cat(mel_outputs, dim=2)

            # Post-Process for Linear Spectrograms
            postnet_out = self.postnet(mel_outputs)
            linear = self.post_proj(postnet_out)
            linear = linear.transpose(1, 2)

            # For easy visualisation
            attn_scores = torch.cat(attn_scores, 1)
            # attn_scores = attn_scores.cpu().data.numpy()
            stop_outputs = torch.cat(stop_outputs, 1)

            return mel_outputs, linear, attn_scores, stop_outputs

        def generate(self, x, speaker_embedding=None, steps=2000):
            self.eval()
            device = next(self.parameters()).device  # use same device as parameters

            batch_size, _  = x.size()

            # Need to initialise all hidden states and pack into tuple for tidyness
            attn_hidden = torch.zeros(batch_size, self.decoder_dims, device=device)
            rnn1_hidden = torch.zeros(batch_size, self.lstm_dims, device=device)
            rnn2_hidden = torch.zeros(batch_size, self.lstm_dims, device=device)
            hidden_states = (attn_hidden, rnn1_hidden, rnn2_hidden)

            # Need to initialise all lstm cell states and pack into tuple for tidyness
            rnn1_cell = torch.zeros(batch_size, self.lstm_dims, device=device)
            rnn2_cell = torch.zeros(batch_size, self.lstm_dims, device=device)
            cell_states = (rnn1_cell, rnn2_cell)

            # Need a <GO> Frame for start of decoder loop
            go_frame = torch.zeros(batch_size, self.n_mels, device=device)

            # Need an initial context vector
            context_vec = torch.zeros(batch_size, self.encoder_dims + self.speaker_embedding_size, device=device)

            # SV2TTS: Run the encoder with the speaker embedding
            # The projection avoids unnecessary matmuls in the decoder loop
            encoder_seq = self.encoder(x, speaker_embedding)
            encoder_seq_proj = self.encoder_proj(encoder_seq)

            # Need a couple of lists for outputs
            mel_outputs, attn_scores, stop_outputs = [], [], []

            # Run the decoder loop
            for t in range(0, steps, self.r):
                prenet_in = mel_outputs[-1][:, :, -1] if t > 0 else go_frame
                mel_frames, scores, hidden_states, cell_states, context_vec, stop_tokens = \
                self.decoder(encoder_seq, encoder_seq_proj, prenet_in,
                             hidden_states, cell_states, context_vec, t, x)
                mel_outputs.append(mel_frames)
                attn_scores.append(scores)
                stop_outputs.extend([stop_tokens] * self.r)
                # Stop the loop when all stop tokens in batch exceed threshold
                if (stop_tokens > 0.5).all() and t > 10: break

            # Concat the mel outputs into sequence
            mel_outputs = torch.cat(mel_outputs, dim=2)

            # Post-Process for Linear Spectrograms
            postnet_out = self.postnet(mel_outputs)
            linear = self.post_proj(postnet_out)


            linear = linear.transpose(1, 2)

            # For easy visualisation
            attn_scores = torch.cat(attn_scores, 1)
            stop_outputs = torch.cat(stop_outputs, 1)

            self.train()

            return mel_outputs, linear, attn_scores

        def init_model(self):
            for p in self.parameters():
                if p.dim() > 1: nn.init.xavier_uniform_(p)

        def get_step(self):
            return self.step.data.item()

        def reset_step(self):
            # assignment to parameters or buffers is overloaded, updates internal dict entry
            self.step = self.step.data.new_tensor(1)

        def log(self, path, msg):
            with open(path, "a") as f:
                print(msg, file=f)

        def load(self, path, optimizer=None):
            # Use device of model params as location for loaded state
            device = next(self.parameters()).device
            checkpoint = torch.load(str(path), map_location=device)
            self.load_state_dict(checkpoint["model_state"])

            if "optimizer_state" in checkpoint and optimizer is not None:
                optimizer.load_state_dict(checkpoint["optimizer_state"])

        def save(self, path, optimizer=None):
            if optimizer is not None:
                torch.save({
                    "model_state": self.state_dict(),
                    "optimizer_state": optimizer.state_dict(),
                }, str(path))
            else:
                torch.save({
                    "model_state": self.state_dict(),
                }, str(path))


        def num_params(self, print_out=True):
            parameters = filter(lambda p: p.requires_grad, self.parameters())
            parameters = sum([np.prod(p.size()) for p in parameters]) / 1_000_000
            if print_out:
                print("Trainable Parameters: %.3fM" % parameters)
            return parameters
    
    
    symbols = [_pad, _eos] + list(_characters) #+ _arpabet
    
    def text_to_sequence(text, cleaner_names):
        """Converts a string of text to a sequence of IDs corresponding to the symbols in the text.

          The text can optionally have ARPAbet sequences enclosed in curly braces embedded
          in it. For example, "Turn left on {HH AW1 S S T AH0 N} Street."

          Args:
            text: string to convert to a sequence
            cleaner_names: names of the cleaner functions to run the text through

          Returns:
            List of integers corresponding to the symbols in the text
        """
        sequence = []

        # Check for curly braces and treat their contents as ARPAbet:
        while len(text):
            m = _curly_re.match(text)
            if not m:
                sequence += _symbols_to_sequence(_clean_text(text, cleaner_names))
                break
            sequence += _symbols_to_sequence(_clean_text(m.group(1), cleaner_names))
            sequence += _arpabet_to_sequence(m.group(2))
            text = m.group(3)

        # Append EOS token
        sequence.append(_symbol_to_id["~"])
        return sequence
    
    def _clean_text(text, cleaner_names):
        class cleaners:
            # Regular expression matching whitespace:
            _whitespace_re = re.compile(r"\s+")

            # List of (regular expression, replacement) pairs for abbreviations:
            _abbreviations = [(re.compile("\\b%s\\." % x[0], re.IGNORECASE), x[1]) for x in [
                ("mrs", "misess"),
                ("mr", "mister"),
                ("dr", "doctor"),
                ("st", "saint"),
                ("co", "company"),
                ("jr", "junior"),
                ("maj", "major"),
                ("gen", "general"),
                ("drs", "doctors"),
                ("rev", "reverend"),
                ("lt", "lieutenant"),
                ("hon", "honorable"),
                ("sgt", "sergeant"),
                ("capt", "captain"),
                ("esq", "esquire"),
                ("ltd", "limited"),
                ("col", "colonel"),
                ("ft", "fort"),
            ]]


            def expand_abbreviations(text):
                for regex, replacement in _abbreviations:
                    text = re.sub(regex, replacement, text)
                return text


            def expand_numbers(text):
                return normalize_numbers(text)


            def lowercase(text):
                """lowercase input tokens."""
                return text.lower()


            def collapse_whitespace(text):
                return re.sub(_whitespace_re, " ", text)


            def convert_to_ascii(text):
                return unidecode(text)


            def basic_cleaners(text):
                """Basic pipeline that lowercases and collapses whitespace without transliteration."""
                text = lowercase(text)
                text = collapse_whitespace(text)
                return text


            def transliteration_cleaners(text):
                """Pipeline for non-English text that transliterates to ASCII."""
                text = convert_to_ascii(text)
                text = lowercase(text)
                text = collapse_whitespace(text)
                return text


            def english_cleaners(text):
                """Pipeline for English text, including number and abbreviation expansion."""
                text = convert_to_ascii(text)
                text = lowercase(text)
                text = expand_numbers(text)
                text = expand_abbreviations(text)
                text = collapse_whitespace(text)
                return text
        
        for name in cleaner_names:
            cleaner = getattr(cleaners, name)
            if not cleaner:
                raise Exception("Unknown cleaner: %s" % name)
            text = cleaner(text)
        return text


    def _symbols_to_sequence(symbols):
        return [_symbol_to_id[s] for s in symbols if _should_keep_symbol(s)]


    def _arpabet_to_sequence(text):
        return _symbols_to_sequence(["@" + s for s in text.split()])


    def _should_keep_symbol(s):
        return s in _symbol_to_id and s not in ("_", "~")
    
    
    
    
    
    
    
            
    
    sample_rate = hparams.sample_rate
    hparams = hparams

    def __init__(self, model_fpath: Path, verbose=True):
        """
        The model isn't instantiated and loaded in memory until needed or until load() is called.

        :param model_fpath: path to the trained model file
        :param verbose: if False, prints less information when using the model
        """
        self.model_fpath = model_fpath
        self.verbose = verbose

        # Check for GPU
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")
        if self.verbose:
            print("Synthesizer using device:", self.device)

        # Tacotron model will be instantiated later on first use.
        self._model = None

    def is_loaded(self):
        """
        Whether the model is loaded in memory.
        """
        return self._model is not None

    def load(self):
        """
        Instantiates and loads the model given the weights file that was passed in the constructor.
        """
        self._model = Tacotron(embed_dims=hparams.tts_embed_dims,
                               num_chars=len(symbols),
                               encoder_dims=hparams.tts_encoder_dims,
                               decoder_dims=hparams.tts_decoder_dims,
                               n_mels=hparams.num_mels,
                               fft_bins=hparams.num_mels,
                               postnet_dims=hparams.tts_postnet_dims,
                               encoder_K=hparams.tts_encoder_K,
                               lstm_dims=hparams.tts_lstm_dims,
                               postnet_K=hparams.tts_postnet_K,
                               num_highways=hparams.tts_num_highways,
                               dropout=hparams.tts_dropout,
                               stop_threshold=hparams.tts_stop_threshold,
                               speaker_embedding_size=hparams.speaker_embedding_size).to(self.device)

        self._model.load(self.model_fpath)
        self._model.eval()

        if self.verbose:
            print("Loaded synthesizer \"%s\" trained to step %d" % (self.model_fpath.name, self._model.state_dict()["step"]))

    def synthesize_spectrograms(self, texts: List[str],
                                embeddings: Union[np.ndarray, List[np.ndarray]],
                                return_alignments=False):
        """
        Synthesizes mel spectrograms from texts and speaker embeddings.

        :param texts: a list of N text prompts to be synthesized
        :param embeddings: a numpy array or list of speaker embeddings of shape (N, 256)
        :param return_alignments: if True, a matrix representing the alignments between the
        characters
        and each decoder output step will be returned for each spectrogram
        :return: a list of N melspectrograms as numpy arrays of shape (80, Mi), where Mi is the
        sequence length of spectrogram i, and possibly the alignments.
        """
        # Load the model on the first request.
        if not self.is_loaded():
            self.load()

        # Preprocess text inputs
        inputs = [text_to_sequence(text.strip(), hparams.tts_cleaner_names) for text in texts]
        if not isinstance(embeddings, list):
            embeddings = [embeddings]

        # Batch inputs
        batched_inputs = [inputs[i:i+hparams.synthesis_batch_size]
                             for i in range(0, len(inputs), hparams.synthesis_batch_size)]
        batched_embeds = [embeddings[i:i+hparams.synthesis_batch_size]
                             for i in range(0, len(embeddings), hparams.synthesis_batch_size)]

        specs = []
        for i, batch in enumerate(batched_inputs, 1):
            if self.verbose:
                print(f"\n| Generating {i}/{len(batched_inputs)}")

            # Pad texts so they are all the same length
            text_lens = [len(text) for text in batch]
            max_text_len = max(text_lens)
            chars = [pad1d(text, max_text_len) for text in batch]
            chars = np.stack(chars)

            # Stack speaker embeddings into 2D array for batch processing
            speaker_embeds = np.stack(batched_embeds[i-1])

            # Convert to tensor
            chars = torch.tensor(chars).long().to(self.device)
            speaker_embeddings = torch.tensor(speaker_embeds).float().to(self.device)

            # Inference
            _, mels, alignments = self._model.generate(chars, speaker_embeddings)
            mels = mels.detach().cpu().numpy()
            for m in mels:
                # Trim silence from end of each spectrogram
                while np.max(m[:, -1]) < hparams.tts_stop_threshold:
                    m = m[:, :-1]
                specs.append(m)

        if self.verbose:
            print("\n\nDone.\n")
        return (specs, alignments) if return_alignments else specs

    @staticmethod
    def load_preprocess_wav(fpath):
        """
        Loads and preprocesses an audio file under the same conditions the audio files were used to
        train the synthesizer.
        """
        wav = librosa.load(str(fpath), hparams.sample_rate)[0]
        if hparams.rescale:
            wav = wav / np.abs(wav).max() * hparams.rescaling_max
        return wav

    @staticmethod
    def make_spectrogram(fpath_or_wav: Union[str, Path, np.ndarray]):
        """
        Creates a mel spectrogram from an audio file in the same manner as the mel spectrograms that
        were fed to the synthesizer when training.
        """
        if isinstance(fpath_or_wav, str) or isinstance(fpath_or_wav, Path):
            wav = Synthesizer.load_preprocess_wav(fpath_or_wav)
        else:
            wav = fpath_or_wav

        mel_spectrogram = audio.melspectrogram(wav, hparams).astype(np.float32)
        return mel_spectrogram

    @staticmethod
    def griffin_lim(mel):
        """
        Inverts a mel spectrogram using Griffin-Lim. The mel spectrogram is expected to have been built
        with the same parameters present in hparams.py.
        """
        return audio.inv_mel_spectrogram(mel, hparams)


def pad1d(x, max_len, pad_value=0):
    return np.pad(x, (0, max_len - len(x)), mode="constant", constant_values=pad_value)

In [6]:
# Utterance
Utterance = namedtuple("Utterance", "name speaker_name wav spec embed partial_embeds synth")
Utterance.__eq__ = lambda x, y: x.name == y.name
Utterance.__hash__ = lambda x: hash(x.name)

NameError: name 'HParams' is not defined